In [175]:
import requests
import pandas as pd
import os

In [1]:
headers = {'User-Agent': "your@email.com"}

In [85]:
response_tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)

In [98]:
ciks_df = pd.DataFrame.from_dict(response_tickers_cik.json()).T
ciks_df["cik_str"] = ciks_df["cik_str"].astype(str).str.zfill(10)

In [99]:
ciks_df

,cik_str,ticker,title
0,0000320193,AAPL,Apple Inc.
1,0000789019,MSFT,MICROSOFT CORP
2,0001652044,GOOGL,Alphabet Inc.
3,0001018724,AMZN,AMAZON COM INC
4,0001318605,TSLA,"Tesla, Inc."
...,...,...,...
12170,0001848558,SHCAU,Spindletop Health Acquisition Corp.
12171,0001854583,ASPAW,"Abri SPAC I, Inc."
12172,0001855555,ENERR,ACCRETION ACQUISITION CORP.
12173,0001855555,ENERW,ACCRETION ACQUISITION CORP.


In [22]:
published_forms_url = "https://data.sec.gov/api/xbrl/companyconcept/CIK{cik_str}/us-gaap/Assets.json"

In [193]:
resurces_url_template = "https://www.sec.gov/Archives/edgar/data/{cik_str}/{accn}"

In [212]:
filled_form_url_template = "https://www.sec.gov/Archives/edgar/data/{cik_str}/{accn}/{end_pt}"

##### Opción 1: Bajarse los html, no es buena idea porque hay que encontrar el nombre del filing y este no sigue ningun estandar.

In [227]:

for r in ciks_df.iterrows():
    company_published_forms = requests.get(published_forms_url.format(cik_str=r[1].cik_str), headers=headers)
    if company_published_forms.status_code==200:
        if 'units' in company_published_forms.json():
            filled_forms = pd.DataFrame.from_dict(list(company_published_forms.json()["units"].values())[0])
            end_dt = filled_forms[filled_forms["fp"] == "FY"][-1:].end.values[0]
            accn = filled_forms[filled_forms["fp"] == "FY"][-1:].accn.values[0]

            resurces_dir_url = resurces_url_template.format(cik_str=r[1].cik_str,
                                                            accn=accn.replace('-', ''))

            resources_list = requests.get(resurces_dir_url, headers=headers).text

            first_occurrence = resources_list.split("{end_dt}.htm".format(end_dt=end_dt.replace('-', '')))[0]
            prefix = first_occurrence.split("/")[-1]
            end_pt = prefix + "{end_dt}.htm".format(end_dt=end_dt.replace('-', ''))

            filled_form_url = filled_form_url_template.format(cik_str=r[1].cik_str, accn=accn.replace('-', ''), end_pt=end_pt)
            filled_form_10k = requests.get(filled_form_url, headers=headers).text

            filename = 'dload_data/' + r[1].ticker + '/' + end_dt + '/' + '10k_filing.htm'
            os.makedirs(os.path.dirname(filename), exist_ok=True)

            with open(filename, "w+") as f:
                f.write(filled_form_10k)

            print(filled_form_url)



https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/aapl-20210925.htm
https://www.sec.gov/Archives/edgar/data/0000789019/000156459021039151/msft-10k_20210630.htm
https://www.sec.gov/Archives/edgar/data/0001652044/000165204422000019/goog-20211231.htm
https://www.sec.gov/Archives/edgar/data/0001018724/000101872422000005/amzn-20211231.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000095017022000796/tsla-20211231.htm
https://www.sec.gov/Archives/edgar/data/0001067983/000156459022007322/brka-10k_20211231.htm
https://www.sec.gov/Archives/edgar/data/0001326801/000132680122000018/fb-20211231.htm
https://www.sec.gov/Archives/edgar/data/0000731766/000073176622000008/unh-20211231.htm
https://www.sec.gov/Archives/edgar/data/0000200406/000020040622000022/jnj-20220102.htm
https://www.sec.gov/Archives/edgar/data/0001045810/000104581022000036/nvda-20220130.htm
https://www.sec.gov/Archives/edgar/data/0000104169/000010416922000012/wmt-20220131.htm
https://www.sec.gov/Archives/

IndexError: index 0 is out of bounds for axis 0 with size 0

##### Opción 2: Bajarse los txt, estos sin embargo si que siguen un estandar así que no perdemos tantos formularios, no hay que realizar llamadas extra para descubrir el nombre del archivo y el trabajo de parseo es el mismo que con los html.

In [229]:
filled_form_url_text_template = "https://www.sec.gov/Archives/edgar/data/{cik_str}/{accn}/{accn_raw}.txt"

for r in ciks_df.iterrows():
    company_published_forms = requests.get(published_forms_url.format(cik_str=r[1].cik_str), headers=headers)
    if company_published_forms.status_code==200:
        if 'units' in company_published_forms.json():
            filled_forms = pd.DataFrame.from_dict(list(company_published_forms.json()["units"].values())[0])
            accn = filled_forms[filled_forms["fp"] == "FY"][-1:].accn.values[0]
            end_dt = filled_forms[filled_forms["fp"] == "FY"][-1:].end.values[0]
            resurces_dir_url = resurces_url_template.format(cik_str=r[1].cik_str,
                                                            accn=accn.replace('-', ''))

            filled_form_url = filled_form_url_text_template.format(cik_str=r[1].cik_str, accn=accn.replace('-', ''), accn_raw=accn)
            filled_form_10k = requests.get(filled_form_url, headers=headers).text

            filename = 'dload_data/' + r[1].ticker + '/' + end_dt + '/' + '10k_filing.txt'
            os.makedirs(os.path.dirname(filename), exist_ok=True)

            with open(filename, "w+") as f:
                f.write(filled_form_10k)

            print(filled_form_url)

https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/0000320193-21-000105.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459021039151/0001564590-21-039151.txt
https://www.sec.gov/Archives/edgar/data/0001652044/000165204422000019/0001652044-22-000019.txt
https://www.sec.gov/Archives/edgar/data/0001018724/000101872422000005/0001018724-22-000005.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000095017022000796/0000950170-22-000796.txt
https://www.sec.gov/Archives/edgar/data/0001067983/000156459022007322/0001564590-22-007322.txt
https://www.sec.gov/Archives/edgar/data/0001326801/000132680122000018/0001326801-22-000018.txt
https://www.sec.gov/Archives/edgar/data/0000731766/000073176622000008/0000731766-22-000008.txt
https://www.sec.gov/Archives/edgar/data/0000200406/000020040622000022/0000200406-22-000022.txt
https://www.sec.gov/Archives/edgar/data/0001045810/000104581022000036/0001045810-22-000036.txt
https://www.sec.gov/Archives/edgar/data/0000104169

KeyboardInterrupt: 